### In this tutorial, we will use DML to merge data into Bigquery
#### MERGE statement
A **MERGE statement** is a DML statement that can combine **INSERT, UPDATE, and DELETE** operations into a single statement and perform the operations atomically.

In [21]:
import os
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/Lenovo/OneDrive/Documents1/Google_Bigquery/premium-botany-431702-u7-498d1c535417.json"
client = bigquery.Client()

##### MERGE
the following query adds new items from the Inventory table to the DetailedInventory table. For items with low inventory, the supply_constrained value is set to true, and comments are added.

In [22]:
# 
QUERY = (
    '''
    MERGE premium-botany-431702-u7.dataset1.DetailedInventory T
    USING premium-botany-431702-u7.dataset1.Inventory S
    ON T.product = S.product
    WHEN NOT MATCHED AND quantity < 20 THEN
      INSERT(product, quantity, supply_constrained, comments)
      VALUES(product, quantity, true, ARRAY<STRUCT<created DATE, comment STRING>>[(DATE('2016-01-01'), 'comment1')])
    WHEN NOT MATCHED THEN
      INSERT(product, quantity, supply_constrained)
      VALUES(product, quantity, false);
      '''
      )
query_job = client.query(QUERY)  # API request

In [23]:
# Perform a query.
QUERY = (
    'SELECT * FROM `premium-botany-431702-u7.dataset1.DetailedInventory` LIMIT 1000;')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

In [24]:
for row in rows:
    print(row)

Row(('countertop microwave', 20, None, [], None), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('oven', 5, False, [], None), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('refrigerator', 10, False, [], None), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('dryer', 30, False, [], None), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('microwave', 20, False, [], None), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('top load washer', 10, False, [{'created': datetime.date(2016, 1, 1), 'comment': 'comment1'}, {'created': datetime.date(2016, 1, 1), 'comment': 'comment1'}], {'color': 'white', 'warranty': '1 year', 'dimensions': None}), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('front load wa

##### MERGE
In the following example, the query merges items from the NewArrivals table into the Inventory table. If an item is already present in Inventory, the query increments the quantity field. Otherwise, the query inserts a new row. Assume that the default value for the supply_constrained column is set to NULL.

In [25]:

QUERY = (
    '''
    MERGE premium-botany-431702-u7.dataset1.Inventory T
    USING premium-botany-431702-u7.dataset1.NewArrivals S
    ON T.product = S.product
    WHEN MATCHED THEN
      UPDATE SET quantity = T.quantity + S.quantity
    WHEN NOT MATCHED THEN
      INSERT (product, quantity) VALUES(product, quantity);
      '''
      )
query_job = client.query(QUERY)  # API request

In [26]:
# Perform a query.
QUERY = (
    'SELECT * FROM `premium-botany-431702-u7.dataset1.Inventory` LIMIT 1000;')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish
for row in rows:
    print(row)

Row(('front load washer', 10, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('refrigerator', 10, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('microwave', 20, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('dishwasher', 20, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('oven', 605, False), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('dryer', 430, False), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('top load washer', 200, False), {'product': 0, 'quantity': 1, 'supply_constrained': 2})


##### MERGE
In the following example, the query increases the quantity of products from warehouse #1 by 20 in the NewArrivals table. The query deletes all other products except for those from warehouse #2.

In [27]:

QUERY = (
    '''
    MERGE premium-botany-431702-u7.dataset1.NewArrivals T
    USING (SELECT * FROM premium-botany-431702-u7.dataset1.NewArrivals WHERE warehouse <> 'warehouse #2') S
    ON T.product = S.product
    WHEN MATCHED AND T.warehouse = 'warehouse #1' THEN
      UPDATE SET quantity = T.quantity + 20
    WHEN MATCHED THEN
      DELETE;
      '''
      )
query_job = client.query(QUERY)  # API request

In [28]:
# Perform a query.
QUERY = (
    'SELECT * FROM `premium-botany-431702-u7.dataset1.NewArrivals` LIMIT 1000;')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish
for row in rows:
    print(row)

Row(('dryer', 200, 'warehouse #2'), {'product': 0, 'quantity': 1, 'warehouse': 2})
Row(('top load washer', 120, 'warehouse #1'), {'product': 0, 'quantity': 1, 'warehouse': 2})


##### MERGE
In the following example, the query replaces all products like '%washer%' in the Inventory table by using the values in the NewArrivals table.

In [29]:

QUERY = (
    '''
    MERGE premium-botany-431702-u7.dataset1.Inventory T
    USING premium-botany-431702-u7.dataset1.NewArrivals S
    ON FALSE
    WHEN NOT MATCHED AND product LIKE '%washer%' THEN
       INSERT (product, quantity) VALUES(product, quantity)
    WHEN NOT MATCHED BY SOURCE AND product LIKE '%washer%' THEN
      DELETE;
'''
)
query_job = client.query(QUERY)  # API request

In [30]:
# Perform a query.
QUERY = (
    'SELECT * FROM `premium-botany-431702-u7.dataset1.Inventory` LIMIT 1000;')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish
for row in rows:
    print(row)

Row(('top load washer', 120, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('microwave', 20, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('refrigerator', 10, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('dryer', 430, False), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('oven', 605, False), {'product': 0, 'quantity': 1, 'supply_constrained': 2})


##### MERGE
In the following example, the query adds a comment in the DetailedInventory table if the product has a lower than average quantity in Inventory table.

In [31]:
QUERY = (
    '''
    MERGE premium-botany-431702-u7.dataset1.DetailedInventory T
    USING premium-botany-431702-u7.dataset1.Inventory S
    ON T.product = S.product
    WHEN MATCHED AND S.quantity < (SELECT AVG(quantity) FROM premium-botany-431702-u7.dataset1.Inventory) THEN
      UPDATE SET comments = ARRAY_CONCAT(comments, ARRAY<STRUCT<created DATE, comment STRING>>[(CAST('2016-02-01' AS DATE), 'comment2')]);
'''
)
query_job = client.query(QUERY)  # API request

In [32]:
# Perform a query.
QUERY = (
    'SELECT * FROM `premium-botany-431702-u7.dataset1.DetailedInventory` LIMIT 1000;')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish
for row in rows:
    print(row)

Row(('countertop microwave', 20, None, [], None), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('dryer', 30, False, [], None), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('oven', 5, False, [], None), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('front load washer', 20, False, [{'created': datetime.date(2016, 1, 1), 'comment': 'comment1'}, {'created': datetime.date(2016, 1, 1), 'comment': 'comment1'}], {'color': 'white', 'warranty': '1 year', 'dimensions': None}), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('dishwasher', 30, False, [{'created': datetime.date(2016, 1, 1), 'comment': 'comment1'}, {'created': datetime.date(2016, 1, 1), 'comment': 'comment1'}], {'color': 'white', 'warranty': '1 year', 'dimensions': None}), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comment

##### MERGE
In the following example, the query increases the inventory of products from the warehouse in CA. The products from other states are deleted, and any product that is not present in the NewArrivals table is unchanged.

In [33]:
QUERY = (
    '''
    MERGE premium-botany-431702-u7.dataset1.Inventory T
    USING (SELECT product, quantity, state FROMpremium-botany-431702-u7.dataset1.NewArrivals t1 JOIN premium-botany-431702-u7.dataset1.Warehouse t2 ON t1.warehouse = t2.warehouse) S
    ON T.product = S.product
    WHEN MATCHED AND state = 'CA' THEN
    UPDATE SET quantity = T.quantity + S.quantity
    WHEN MATCHED THEN
      DELETE;
'''
)
query_job = client.query(QUERY)  # API request

In [34]:
# Perform a query.
QUERY = (
    'SELECT * FROM `premium-botany-431702-u7.dataset1.Inventory` LIMIT 1000;')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish
for row in rows:
    print(row)

Row(('top load washer', 120, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('microwave', 20, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('refrigerator', 10, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('dryer', 430, False), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('oven', 605, False), {'product': 0, 'quantity': 1, 'supply_constrained': 2})


##### MERGE
In the following example, a runtime error is returned because the query attempts to update a target table when the source contains more than one matched row. To resolve the error, you need to change the merge_condition or search_condition to avoid duplicate matches in the source.

In [35]:
QUERY = (
    '''
    MERGE premium-botany-431702-u7.dataset1.Inventory T
    USING premium-botany-431702-u7.dataset1.NewArrivals S
    ON T.product = S.product
    WHEN MATCHED THEN
      UPDATE SET quantity = T.quantity + S.quantity;
'''
)
query_job = client.query(QUERY)  # API request

In [36]:
# Perform a query.
QUERY = (
    'SELECT * FROM `premium-botany-431702-u7.dataset1.Inventory` LIMIT 1000;')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish
for row in rows:
    print(row)

Row(('dryer', 630, False), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('top load washer', 240, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('microwave', 20, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('refrigerator', 10, None), {'product': 0, 'quantity': 1, 'supply_constrained': 2})
Row(('oven', 605, False), {'product': 0, 'quantity': 1, 'supply_constrained': 2})


##### MERGE
In the following example, all of the products in the NewArrivals table are replaced with values from the subquery. The INSERT clause does not specify column names for either the target table or the source subquery.

In [37]:
QUERY = (
    '''
    MERGE premium-botany-431702-u7.dataset1.NewArrivals
USING (SELECT * FROM UNNEST([('microwave', 10, 'warehouse #1'),
                             ('dryer', 30, 'warehouse #1'),
                             ('oven', 20, 'warehouse #2')]))
ON FALSE
WHEN NOT MATCHED THEN
  INSERT ROW
WHEN NOT MATCHED BY SOURCE THEN
  DELETE;
'''
)
query_job = client.query(QUERY)  # API request

In [38]:
# Perform a query.
QUERY = (
    'SELECT * FROM `premium-botany-431702-u7.dataset1.NewArrivals` LIMIT 1000;')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish
for row in rows:
    print(row)

Row(('dryer', 30, 'warehouse #1'), {'product': 0, 'quantity': 1, 'warehouse': 2})
Row(('microwave', 10, 'warehouse #1'), {'product': 0, 'quantity': 1, 'warehouse': 2})
Row(('oven', 20, 'warehouse #2'), {'product': 0, 'quantity': 1, 'warehouse': 2})


##### UPDATE statement using join between three tables
The following example sets supply_constrained to true for all products from NewArrivals where the warehouse location is in 'WA' state.

In [39]:
QUERY = (
    '''
    UPDATE premium-botany-431702-u7.dataset1.DetailedInventory
    SET supply_constrained = true
    FROM premium-botany-431702-u7.dataset1.NewArrivals, premium-botany-431702-u7.dataset1.Warehouse
    WHERE DetailedInventory.product = NewArrivals.product AND
          NewArrivals.warehouse = Warehouse.warehouse AND
          Warehouse.state = 'WA';
'''
)
query_job = client.query(QUERY)  # API request

In [40]:
# Perform a query.
QUERY = (
    'SELECT * FROM `premium-botany-431702-u7.dataset1.DetailedInventory` LIMIT 1000;')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish
for row in rows:
    print(row)

Row(('countertop microwave', 20, None, [], None), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('dryer', 30, False, [], None), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('oven', 5, False, [], None), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('front load washer', 20, False, [{'created': datetime.date(2016, 1, 1), 'comment': 'comment1'}, {'created': datetime.date(2016, 1, 1), 'comment': 'comment1'}], {'color': 'white', 'warranty': '1 year', 'dimensions': None}), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comments': 3, 'specifications': 4})
Row(('dishwasher', 30, False, [{'created': datetime.date(2016, 1, 1), 'comment': 'comment1'}, {'created': datetime.date(2016, 1, 1), 'comment': 'comment1'}], {'color': 'white', 'warranty': '1 year', 'dimensions': None}), {'product': 0, 'quantity': 1, 'supply_constrained': 2, 'comment